In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("red-wine.csv")

In [13]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
from tensorflow import keras
from tensorflow.keras import layers ,callbacks

In [5]:
from sklearn.compose import make_column_transformer , make_column_selector
from sklearn.preprocessing import StandardScaler

In [6]:
ss= StandardScaler()

In [7]:
transform = make_column_transformer(
    (ss,make_column_selector(dtype_exclude='object'))
)

In [14]:
X = transform.fit_transform(df.drop(columns='quality'))
y = np.log(df['quality'])

In [20]:
pd.DataFrame(X,columns=transform.feature_names_in_).head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,-0.528360,0.961877,-1.391472,-0.453218,-0.243707,-0.466193,-0.379133,0.558274,1.288643,-0.579207,-0.960246
1,-0.298547,1.967442,-1.391472,0.043416,0.223875,0.872638,0.624363,0.028261,-0.719933,0.128950,-0.584777
2,-0.298547,1.297065,-1.186070,-0.169427,0.096353,-0.083669,0.229047,0.134264,-0.331177,-0.048089,-0.584777
3,1.654856,-1.384443,1.484154,-0.453218,-0.264960,0.107592,0.411500,0.664277,-0.979104,-0.461180,-0.584777
4,-0.528360,0.961877,-1.391472,-0.453218,-0.243707,-0.466193,-0.379133,0.558274,1.288643,-0.579207,-0.960246


In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

In [76]:
early_stopping = callbacks.EarlyStopping(
    min_delta=0.1, # minimium amount of change to count as an improvement
    patience=2, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [77]:
model = keras.Sequential([
    layers.Dense(units = 10 , activation='relu',input_shape=[11]),
    layers.Dense(units = 5,activation='relu'),
    layers.Dense(units=1)
])

C:\Users\sudhi\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [78]:
model.compile(optimizer='adam',loss='mae')

In [79]:
model.fit(X_train,y_train,batch_size=512 , epochs=50,
         callbacks=[early_stopping], # put your callbacks in a list
          verbose=0)

C:\Users\sudhi\anaconda3\lib\site-packages\keras\src\callbacks\early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


In [80]:
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 741us/step


In [81]:
from sklearn.metrics import mean_absolute_percentage_error,mean_absolute_error

In [82]:
print(" train MAE :: ",mean_absolute_error(y_train,train_pred))
print(" test MAE :: ",mean_absolute_error(y_test,test_pred))

 train MAE ::  0.3945365885859223
 test MAE ::  0.3887382133501376


In [83]:
print(" train MAPE :: ",mean_absolute_percentage_error(y_train,train_pred))
print(" test MAPE :: ",mean_absolute_percentage_error(y_test,test_pred))

 train MAPE ::  0.23247182387440102
 test MAPE ::  0.22609535904459466
